# Importação dos dados

In [ ]:
from google.colab import files
import pandas as pd
# Abre a janela para selecionar o arquivo
uploaded = files.upload()
#Carregando a matriz de dados de treinamento
x_train= pd.read_csv('x_train.csv')

In [ ]:
uploaded = files.upload()
y_train= pd.read_csv('y_train.csv')

In [ ]:
uploaded = files.upload()
x_test= pd.read_csv('x_test.csv')

In [ ]:
uploaded = files.upload()
y_test= pd.read_csv('y_test.csv')

# 1. Treinamento do modelo e teste

## 1.1. Treinamento

In [ ]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, cross_val_predict
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd

# Dicionário de modelos e hiperparâmetros
modelos_parametros = {
    'Random Forest': {
        'modelo': RandomForestClassifier(),
        'parametros': {
            'n_estimators': [100, 200],
            'max_depth': [None, 10, 20]
        }
    },
    'SVM': {
        'modelo': SVC(),
        'parametros': {
            'C': [0.1, 1, 10],
            'kernel': ['linear', 'rbf']
        }
    },
    'KNN': {
        'modelo': KNeighborsClassifier(),
        'parametros': {
            'n_neighbors': [3, 5, 7],
            'metric': ['euclidean', 'manhattan']
        }
    },
    'XGBoost': {
        'modelo': XGBClassifier(use_label_encoder=False, eval_metric='mlogloss'),
        'parametros': {
            'n_estimators': [100, 200],
            'learning_rate': [0.01, 0.1],
            'max_depth': [3, 6]
        }
    },
    'Decision Tree': {
        'modelo': DecisionTreeClassifier(),
        'parametros': {
            'max_depth': [None, 5, 10],
            'criterion': ['gini', 'entropy']
        }
    },
    'Logistic Regression': {
        'modelo': LogisticRegression(max_iter=1000),
        'parametros': {
            'C': [0.1, 1, 10],
            'penalty': ['l2']
        }
    },
    'Gradient Boosting': {
        'modelo': GradientBoostingClassifier(),
        'parametros': {
            'n_estimators': [100, 200],
            'learning_rate': [0.01, 0.1],
            'max_depth': [3, 6]
        }
    }
}

# DataFrame para armazenar os resultados
resultados = []

# Loop de treinamento e avaliação
for nome, config in modelos_parametros.items():
    print(f"\n🔧 Treinando: {nome}")
    grid = GridSearchCV(config['modelo'], config['parametros'], cv=10, scoring='accuracy', n_jobs=-1)
    grid.fit(x_train, y_train)

    # Previsões com validação cruzada (k=10)
    y_pred = cross_val_predict(grid.best_estimator_, x_train, y_train, cv=10)

    # Métricas
    acc = accuracy_score(y_train, y_pred)
    prec = precision_score(y_train, y_pred, average='weighted', zero_division=0)
    rec = recall_score(y_train, y_pred, average='weighted')
    f1 = f1_score(y_train, y_pred, average='weighted')

    resultados.append({
        'Modelo': nome,
        'Acurácia': acc,
        'Precisão': prec,
        'Recall': rec,
        'F1-Score': f1,
        'Melhores Hiperparâmetros': grid.best_params_
    })

# Criar DataFrame de resultados
df_resultados = pd.DataFrame(resultados)
df_resultados = df_resultados.sort_values(by='F1-Score', ascending=False)

# Exibir a tabela
print("\n📊 Resultados dos Modelos (validação cruzada no treino):")
print(df_resultados[['Modelo', 'Acurácia', 'Precisão', 'Recall', 'F1-Score']].round(3))

# Exibir hiperparâmetros
print("\n🔍 Melhores Hiperparâmetros:")
for i, row in df_resultados.iterrows():
    print(f"{row['Modelo']}: {row['Melhores Hiperparâmetros']}")
